In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
inputPath = "D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

['False' 'True']
2


In [3]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



for i in filesArray:
    print(i)

1248
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_191958.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192022.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192026.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192043.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192126.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192132.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192140.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192153.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192209.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\D

In [4]:
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1, 0]
    else:
        label = [0, 1]
    labels.append(label)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)


In [7]:
model = Sequential()
model.add(L.Input((1000, 1), name='input_accel', dtype="float32"))
'''
# Rebuild Top
model.add(L.Conv1D(32,3, input_shape = (1000,1)))
model.add(L.Conv1D(8,8, input_shape = (1000,1)))
model.add(L.Dense(128, activation="relu"))
model.add(L.BatchNormalization())
model.add(L.Dense(32, activation="relu"))
model.add(L.Dropout(0.25))
model.add(L.GlobalMaxPooling1D())
model.add(L.Dense(2, activation="softmax"))
'''
model.add(L.Conv1D(128,2, activation="relu", input_shape = (1000,1)))
model.add(L.Dropout(0.2))
model.add(L.Conv1D(256,2, activation="relu"))
model.add(L.GlobalMaxPooling1D())
model.add(L.Dropout(0.2))
model.add(L.Flatten())
model.add(L.Dense(128, activation="relu"))
model.add(L.Dense(2, activation="softmax"))

model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
# model.build((235, 4001, 1))
print(model.summary())


df_array = np.array(df)
labels_array = np.array(labels)

model.fit(df_array, labels_array, shuffle=1, epochs=20, validation_split=0.15, batch_size = 1)




Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 999, 128)          384       
                                                                 
 dropout_4 (Dropout)         (None, 999, 128)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 998, 256)          65792     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 flatten_2 (Flatten)         (None, 256)               0         
                                                      

In [8]:
model.evaluate(df_array, labels_array, batch_size=1)

1248/1248 [==============================] - 2s 1ms/step - loss: 0.3283 - categorical_accuracy: 0.8750


[0.3283369243144989, 0.875]